In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export 
from exp.nb_02 import * 
import torch.nn.functional as F

## Initial Setup 

In [5]:
mpl.rcParams['image.cmap'] = 'gray'

In [6]:
x_train,y_train,x_valid,y_valid = get_data()

In [7]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [9]:
class Model(nn.Module):
    def __init__(self, n_ih, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_ih, nh), nn.ReLU(), nn.Linear(nh, n_out)]
    
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [10]:
model = Model(m, nh, 10)

In [11]:
pred = model(x_train)

### Cross Entropy Loss

First, we will need to compute the softmax of our activations. This is defined by:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{e^{x_{0}} + e^{x_{1}} + \cdots + e^{x_{n-1}}}$$

or more concisely:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum_{0 \leq j \leq n-1} e^{x_{j}}}$$ 

In practice, we will need the log of the softmax when we calculate the loss.

In [14]:
def log_softmax(x): return (x.exp()/(x.exp().sum(-1, keepdim=True))).log()

In [15]:
sm_pred = log_softmax(pred)

The cross entropy loss for some target $x$ and some prediction $p(x)$ is given by:

$$ -\sum x\, \log p(x) $$

But since our $x$s are 1-hot encoded, this can be rewritten as $-\log(p_{i})$ where i is the index of the desired target.

This can be done using numpy-style [integer array indexing](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html#integer-array-indexing). Note that PyTorch supports all the tricks in the advanced indexing methods discussed in that link.

In [16]:
y_train[:3]

tensor([5, 0, 4])

In [17]:
sm_pred[[0,1,2], [5, 0,4]]

tensor([-2.2603, -2.3796, -2.3349], grad_fn=<IndexBackward>)

In [18]:
y_train.shape[0]

50000

In [20]:
def nll(input, target): return -input[range(target.shape[0]), target].mean()

In [21]:
loss = nll(sm_pred, y_train)

In [22]:
loss

tensor(2.3048, grad_fn=<NegBackward>)

Note that the formula 

$$\log \left ( \frac{a}{b} \right ) = \log(a) - \log(b)$$ 

gives a simplification when we compute the log softmax, which was previously defined as `(x.exp()/(x.exp().sum(-1,keepdim=True))).log()`

In [23]:
def log_softmax(x): return x - x.exp().sum(-1, keepdim=True).log()

In [24]:
test_near(nll(log_softmax(pred), y_train), loss)

Then, there is a way to compute the log of the sum of exponentials in a more stable way, called the [LogSumExp trick](https://en.wikipedia.org/wiki/LogSumExp). The idea is to use the following formula:

$$\log \left ( \sum_{j=1}^{n} e^{x_{j}} \right ) = \log \left ( e^{a} \sum_{j=1}^{n} e^{x_{j}-a} \right ) = a + \log \left ( \sum_{j=1}^{n} e^{x_{j}-a} \right )$$

where a is the maximum of the $x_{j}$.

In [ ]:
def logsumexp(x):
    